In [252]:
#import libraries
from scipy import stats
import uproot
import matplotlib as mpl
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LinearRegression


In [ ]:

#Creating the noise file 
def open_noise(filename) :
    noise0=uproot.open(filename) #using uproot to open the root file 
    noise1=noise0['pixTree']
    noise=pd.DataFrame(noise1['fData'].array(library="np"))
    noise=noise[noise.duplicated(subset=['row','col'])==True]
    noise.drop_duplicates(subset=['row','col'],inplace=True)  #one consider a pixel tobe noisy xhen on a short run (<1minute), with no source it declares a hit more than one time.
    X_noise=np.array(noise.col+((noise.chipId-4)*1024)) #changing the coordinates 
    Y_noise=np.array(noise.row)
    return X_noise,Y_noise

#put in the Noise data 
Noise1=open_noise('../Noise1')
Noise2=open_noise('../Noise2')
Noise3=open_noise('../Noise3')
Noise4=open_noise('../Noise4')


In [ ]:
#open the root file and getting the tree
def open_file (filename,id):
    file=uproot.open(filename)
    file1=file['pixTree']
    Data=file1['fData'].array(library="np")
    data=pd.DataFrame(Data)
    data['ldr']=id
    data.col=((data.chipId-4)*1024)+data.col
    return data

data=open_file("../Data1")
data2=open_file("../Data2")
data3=open_file("../Data3")
data4=open_file("../Data4")

data=pd.concat([data,data2])
data=pd.concat([data,data3])
data=pd.concat([data,data4])





fig, axes = plt.subplots(4)

display(data) #show the table with the different entries
axes[0].hist2d(data[data.ldr==1].col,data[data.ldr==1].row,bins=100,density=True) #plotting the data as a 2D histogram (at this point data from ladder 2 and 4 are flipped)
axes[1].hist2d(data[data.ldr==2].col,data[data.ldr==2].row,bins=100,density=True)
axes[2].hist2d(data[data.ldr==3].col,data[data.ldr==3].row,bins=100,density=True)
axes[3].hist2d(data[data.ldr==4].col,data[data.ldr==4].row,bins=100,density=True)
plt.show()

In [285]:
#remove noisy pixels 
def remove_noise (noise,data) :
        n_0=data[data.col.isin(noise[0]) & data.row.isin(noise[1])].index
        if len(n_0)>0:
            data.drop(index=n_0,inplace=True)
        return data 


data_filtered=remove_noise(Noise1,data[data.ldr==1])
data_filtered=pd.concat([data_filtered,remove_noise(Noise2,data[data.ldr==2])])
data_filtered=pd.concat([data_filtered,remove_noise(Noise2,data[data.ldr==3])])
data_filtered=pd.concat([data_filtered,remove_noise(Noise2,data[data.ldr==4])])


In [ ]:

#Displaying the filtered data (Monitoring the experiment)
display(data_filtered)
fig, axes = plt.subplots(4)
axes[0].hist2d(data_filtered[(data_filtered.ldr==1)&(data_filtered.chipId==8)].col,data_filtered[(data_filtered.ldr==1)&(data_filtered.chipId==8)].row,bins=[600,300],norm='symlog')
axes[1].hist2d(data_filtered[(data_filtered.ldr==2)&(data_filtered.chipId==8)].col,data_filtered[(data_filtered.ldr==2)&(data_filtered.chipId==8)].row,bins=[600,300],norm='symlog')
axes[2].hist2d(data_filtered[(data_filtered.ldr==3)&(data_filtered.chipId==8)].col,data_filtered[(data_filtered.ldr==3)&(data_filtered.chipId==8)].row,bins=[600,300],norm='symlog')
axes[3].hist2d(data_filtered[(data_filtered.ldr==4)&(data_filtered.chipId==8)].col,data_filtered[(data_filtered.ldr==4)&(data_filtered.chipId==8)].row,bins=[600,300],norm='symlog')
plt.show()

In [ ]:
#flipping the data from the even plane in y 
def flip_data_row (data) :
    data.row=524-data.row
    return data

data_filtered[(data_filtered.ldr==2)]=flip_data_row(data_filtered[(data_filtered.ldr==2)])
data_filtered[(data_filtered.ldr==4)]=flip_data_row(data_filtered[(data_filtered.ldr==4)])


In [ ]:
#Alignement of the telescope 
def track (Pos_1,Pos_2,z) :
    x=((z-Pos_1[2])/(Pos_2[2]-Pos_1[2]))+Pos_1[0]
    y=((z-Pos_1[2])/(Pos_2[2]-Pos_1[2]))+Pos_1[1]
    return [x,y]

#for this first alignement, we just consider the mean in x and y of the hits and we built track 2 by 2, then moving the other plane to be on this track
def alignement_telescope (DATA,Z) :
    for i in range (max(DATA.ldr)) :
        for j in range (max(DATA.ldr)-1,i,-1) :
            for k in range (max(DATA.ldr)) :
                if k!=j and k!=i :
                    Moved=track([np.mean(DATA[DATA.ldr==i+1].col),np.mean(DATA[DATA.ldr==i+1].row),Z[i]],[np.mean(DATA[DATA.ldr==j+1].col),np.mean(DATA[DATA.ldr==j+1].row),Z[j]],Z[k])
                    DATA.loc[DATA.ldr==k+1,'row']=DATA[DATA.ldr==k+1].row+(Moved[1]-np.mean(DATA[DATA.ldr==k+1].row))
                    DATA.loc[DATA.ldr==k+1,'col']=DATA[DATA.ldr==k+1].col+(Moved[0]-np.mean(DATA[DATA.ldr==k+1].col))
    return DATA


Z=[0,10.6,145,155.6]#coordinates of the differents planes of BANCO
data_aligned=alignement_telescope(data_filtered[data_filtered.chipId==8],Z)
display(data_aligned)

In [ ]:
#Plotting superposed (to see if planes are aligned)
fig, axes = plt.subplots()
plt.hist2d(data_aligned[data_aligned.ldr==1].col,data_aligned[data_aligned.ldr==1].row,bins=[600,300],norm='symlog',cmap='Greens')
plt.hist2d(data_aligned[data_aligned.ldr==2].col,data_aligned[data_aligned.ldr==2].row,bins=[600,300],norm='symlog',cmap='Blues',alpha=0.2)
plt.hist2d(data_aligned[data_aligned.ldr==3].col,data_aligned[data_aligned.ldr==3].row,bins=[600,300],norm='symlog',cmap='Reds',alpha=0.2)
plt.hist2d(data_aligned[data_aligned.ldr==4].col,data_aligned[data_aligned.ldr==4].row,bins=[600,300],norm='symlog',cmap='viridis',alpha=0.2,cmin=1)
plt.axis('scaled')
plt.show()

In [294]:
#Clustering with Agglomerative Clustering (this method use a clustering method based on dichotomy )

def Clustering (data) :
    n=0
    Cluster=[]
    L=[]
    for i in range(max(data.trgNum)+1) :
        if data[data.trgNum==i].empty ==False :
            if (i in L) ==False :
                L.append(i)
                data_trgNum= data[data.trgNum==i] 
                if len(data_trgNum)!=1 :
                    X=data_trgNum[['row','col']].to_numpy()
                    clustering = AgglomerativeClustering(n_clusters=None,distance_threshold=2,metric='euclidean').fit(X)
                    labels = clustering.labels_ 
                    for i in labels :
                        Cluster.append(i+n)
                    n=max(labels)+1+n
                else :
                    Cluster.append(n)
                    n+=1
    return Cluster

Clusters=[]
for i in range (max(data_aligned.ldr)):
    Clusters.append(Clustering(data_aligned[data_aligned.ldr==i+1]))


In [295]:
#Setting the clusters to the right data

data_clusterized=data_aligned.copy()
data_clusterized['cluster']=0
for i in range(max(data_clusterized.ldr)):
    data_clusterized.loc[data_clusterized.ldr==i+1,'cluster']=np.array(Clusters[i])+max(data_clusterized.cluster)

In [296]:

#computing barycenter

def reduce_by_barycenter (data) :
    data['Xbar'] = 0
    data['Ybar'] = 0
    for i in range (1,data.cluster.max()+1) :
        data.loc[data['cluster']==i,'Xbar']=data[data['cluster']==i].col.mean()
        data.loc[data['cluster']==i,'Ybar']=data[data['cluster']==i].row.mean()
    B=data.drop_duplicates(subset=['Xbar','Ybar','cluster'],keep='first',inplace=False)
    return B


data_reduced=reduce_by_barycenter (data_clusterized)



In [297]:
#fit the data 
def chi2 (xexp,xfit): #computing chi2
    chi2 =0 
    for i in range (len(xexp)):
        chi2+=((xexp[i]-xfit[i])**2)/xexp[i]
    return chi2

def tracking (X,Y,Z0,t) : #getting the tracks we want and there residuals 
    slope_X, intercept_X= stats.linregress(Z0, X)
    slope_Y, intercept_Y= stats.linregress(Z0, Y)
    Mean_Residuals_X=np.mean((1-slope_X)*X-intercept_X)
    Mean_Residuals_Y=np.mean((1-slope_Y)*Y-intercept_Y)
    return pd.DataFrame({'trgNum':t,'Mean_Residuals_X':Mean_Residuals_X,'Mean_Residuals_Y':Mean_Residuals_Y,'slope_X':slope_X,'intercept_X':intercept_X,'slope_Y':slope_Y,'intercept_Y':intercept_Y})



In [ ]:

#Alignement from the tracking : In this more precise alignement technique we compute all the residuals from a track (3 points fitting, distance from the track of the 4th one)

def precise_alignement (data,Z0) :
    Distances_X=[[],[],[],[]]
    Distances_Y=[[],[],[],[]]
    for t in range(max(data.trgNum)) :
        if (len(data[data.trgNum==t])==max(data.ldr) and data[data.trgNum==t].ldr.nunique() == data[data.trgNum==t].ldr.size):
            for k in range(max(data.ldr)):
                X=[]
                Y=[]
                Z=[]
                for j in range (max(data.ldr)):
                    if j!=k:
                        X=[data.at[data[(data.trgNum==t)&(data.ldr==j+1)],'Xbar']]
                        Y=[data.at[data[(data.trgNum==t)&(data.ldr==j+1)],'Ybar']]
                        Z.append(Z0[j])
                t=tracking (X,Y,Z)
                distance_x=T['slope_X']*Z0[k]+T['intercept_X']-[data.at[data[(data.trgNum==t)&(data.ldr==k+1)],'Xbar']]
                distance_y=T['slope_Y']*Z0[k]+T['intercept_Y']-[data.at[data[(data.trgNum==t)&(data.ldr==k+1)],'Ybar']]
                Distances_X[k].append(distance_x)
                Distances_Y[k].append(distance_y)
    return pd.DataFrame({'dx':Distances_X,'dy':Distances_Y})



Z=[0,10.6,145,155.6]
Distances=precise_alignement(data_reduced,Z)
print('x',np.mean(Distances['dx'][0]),np.mean(Distances['dx'][1]),np.mean(Distances['dx'][2]),np.mean(Distances['dx'][3]),'Y',np.mean(Distances['dy'][0]),np.mean(Distances['dy'][1]),np.mean(Distances['dy'][2]),np.mean(Distances['dy'][3]))

In [ ]:
#moving from the correction that we had previously
def move (data,X,Y):
    for ldr in range(max(data.ldr)) :
        data.loc[data.ldr==ldr+1,'row']=data.row+np.mean(X[ldr])
        data.loc[data.ldr==ldr+1,'col']=data.col+np.mean(Y[ldr])
    return data


data_aligned_2=move(data,Distances['dx'],Distances['dy'])



In [233]:
#computing all the tracks for a run 

def make_tracks (data,Z) :
    Tracks=pd.DataFrame({{'trgNum':[],'Mean_Residuals_X':[],'Mean_Residuals_Y':[],'slope_X':[],'intercept_X':[],'slope_Y':[],'intercept_Y':[]}})
    for t in range(max(data.trgNum)) :
        if (len(data[data.trgNum==t])==max(data.ldr) and data[data.trgNum==t].ldr.nunique() == data[data.trgNum==t].ldr.size):
            X=[data.at[data[(data.trgNum==t)&(data.ldr==i+1)],'Xbar'] for i in range (max(data.ldr))]
            Y=[data.at[data[(data.trgNum==t)&(data.ldr==i+1)],'Ybar'] for i in range (max(data.ldr))]
            T=tracking (X,Y,Z)
            pd.concat(Tracks,T)
    return Tracks


Tracks=make_tracks(data_aligned,Z)